# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das Bibliotecas

In [1]:
!pip install skorch==0.7.0

In [2]:
!pip install torch==1.4.0

In [3]:
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F

from skorch import NeuralNetBinaryClassifier
from sklearn.model_selection import GridSearchCV
torch.__version__

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


'1.4.0'

## Etapa 2: Base de dados

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
previsores = pd.read_csv('/content/entradas_breast.csv')
classes = pd.read_csv('/content/saidas_breast.csv')

In [6]:
previsores = np.array(previsores, dtype='float32')
classes = np.array(classes, dtype='float32').squeeze(1)

In [7]:
previsores.shape, classes.shape

((569, 30), (569,))

## Etapa 3: Classe para estrutura da rede neural

In [8]:
class cls_torch(nn.Module):

  def __init__(self, activation, neurons, initializer):
    super().__init__()

    # 30->16->16->1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

## Etapa 4: Skorch

In [9]:
cls_sklearn = NeuralNetBinaryClassifier(module=cls_torch,
                                        lr = 0.001,
                                        optimizer__weight_decay = 0.0001,
                                        train_split=False)

## Etapa 5: Tuning dos parâmetros

In [11]:
params = {'batch_size': [10], # 30],
          'max_epochs': [100], #50],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],
          'module__activation':[F.relu, F.tanh],
          'module__neurons': [8,16],
          'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_]}

In [12]:
params

{'batch_size': [10],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'max_epochs': [100],
 'module__activation': [<function torch.nn.functional.relu>,
  <function torch.nn.functional.tanh>],
 'module__initializer': [<function torch.nn.init.uniform_>,
  <function torch.nn.init.normal_>],
 'module__neurons': [8, 16],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD]}

In [13]:
grid_search = GridSearchCV(estimator=cls_sklearn, param_grid = params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classes)

  epoch    train_loss     dur
-------  ------------  ------
      1       10.3130  0.1149
      2       10.3130  0.0317
      3       10.3130  0.0331
      4       10.3130  0.0274
      5       10.3130  0.0316
      6       10.3130  0.0386
      7       10.3130  0.0374
      8       10.3130  0.0373
      9       10.3130  0.0331
     10       10.3130  0.0300
     11       10.3130  0.0388
     12       10.3130  0.0409
     13       10.3130  0.0297
     14       10.3130  0.0359
     15       10.3130  0.0372
     16       10.3130  0.0378
     17       10.3130  0.0396
     18       10.3130  0.0297
     19       10.3130  0.0363
     20       10.3130  0.0386
     21       10.3130  0.0388
     22       10.3130  0.0360
     23       10.3130  0.0353
     24       10.3130  0.0291
     25       10.3130  0.0291
     26       10.3130  0.0284
     27       10.3130  0.0285
     28       10.3130  0.0302
     29       10.3130  0.0293
     30       10.3130  0.0326
     31       10.3130  0.0289
     32   

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


      5        1.3730  0.0463
      6        1.2963  0.0406
      7        1.2054  0.0395
      8        1.0817  0.0381
      9        0.9130  0.0383
     10        0.7607  0.0386
     11        0.6957  0.0425
     12        0.6777  0.0401
     13        0.6734  0.0408
     14        0.6721  0.0391
     15        0.6714  0.0383
     16        0.6709  0.0303
     17        0.6706  0.0364
     18        0.6703  0.0390
     19        0.6701  0.0390
     20        0.6699  0.0385
     21        0.6697  0.0386
     22        0.6696  0.0427
     23        0.6694  0.0390
     24        0.6693  0.0375
     25        0.6692  0.0370
     26        0.6692  0.0480
     27        0.6691  0.0399
     28        0.6690  0.0399
     29        0.6690  0.0393
     30        0.6689  0.0419
     31        0.6689  0.0302
     32        0.6688  0.0303
     33        0.6688  0.0306
     34        0.6687  0.0312
     35        0.6687  0.0305
     36        0.6687  0.0290
     37        0.6686  0.0322
     38   

In [14]:
best_params = grid_search.best_params_
best_acc = grid_search.best_score_

In [15]:
best_params

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu>,
 'module__initializer': <function torch.nn.init.uniform_>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [16]:
best_acc

0.7524153694094391